<a href="https://colab.research.google.com/github/MIT-LCP/2019_tokyo_datathon/blob/master/02_severity_of_illness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# eICU Collaborative Research Database

# Notebook 2: Severity of illness

This notebook introduces high level admission details relating to a single patient stay, using the following tables:

- patient
- admissiondx
- apacheapsvar
- apachepredvar
- apachepatientresult



## Load libraries and connect to the database

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
# authenticate
auth.authenticate_user()

In [0]:
# Set up environment variables
project_id='datathonjapan2019'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [0]:
# Helper function to read data from BigQuery into a DataFrame.
def run_query(query):
    return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, 
                            configuration={'query':{'useLegacySql': False}})

## Selecting a single patient stay¶

As we have seen, the patient table includes general information about the patient admissions (for example, demographics, admission and discharge details). See: http://eicu-crd.mit.edu/eicutables/patient/

## Questions

Use your knowledge from the previous notebook and the online documentation (http://eicu-crd.mit.edu/) to answer the following questions:

- Which column in the patient table is distinct for each stay in the ICU (similar to `icustay_id` in MIMIC-III)?
- Which column is unique for each patient (similar to `subject_id` in MIMIC-III)?

In [0]:
# view distinct ids
query = """
SELECT DISTINCT(patientunitstayid)
FROM `physionet-data.eicu_crd_demo.patient`
"""

run_query(query)


In [0]:
# select a single ICU stay
patientunitstayid = <your_id_here>

In [0]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.patient`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

patient = run_query(query)

In [0]:
patient

## Questions

- Which type of unit was the patient admitted to? Hint: Try `patient['unittype']` or `patient.unittype`
- What year was the patient discharged from the ICU? Hint: You can view the table columns with `patient.columns`
- What was the status of the patient upon discharge from the unit?

## The admissiondx table

The `admissiondx` table contains the primary diagnosis for admission to the ICU according to the APACHE scoring criteria. For more detail, see: http://eicu-crd.mit.edu/eicutables/admissiondx/

In [0]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.admissiondx`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

admissiondx = run_query(query)

In [0]:
# View the columns in this data
admissiondx.columns

In [0]:
# View the data
admissiondx.head()

In [0]:
# Set the display options to avoid truncating the text
pd.set_option('display.max_colwidth', -1)
admissiondx.admitdxpath

## Questions

- What was the primary reason for admission?
- How soon after admission to the ICU was the diagnoses recorded in eCareManager? Hint: The `offset` columns indicate the time in minutes after admission to the ICU. 

## The apacheapsvar table

The apacheapsvar table contains the variables used to calculate the Acute Physiology Score (APS) III for patients. APS-III is an established method of summarizing patient severity of illness on admission to the ICU, taking the "worst" observations for a patient in a 24 hour period.

The score is part of the Acute Physiology Age Chronic Health Evaluation (APACHE) system of equations for predicting outcomes for ICU patients. See: http://eicu-crd.mit.edu/eicutables/apacheApsVar/

In [0]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apacheapsvar`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apacheapsvar = run_query(query)

In [0]:
apacheapsvar.head()

## Questions

- What was the 'worst' heart rate recorded for the patient during the scoring period?
- Was the patient oriented and able to converse normally on the day of admission? (hint: the verbal element refers to the Glasgow Coma Scale).

# apachepredvar table

The apachepredvar table provides variables underlying the APACHE predictions. Acute Physiology Age Chronic Health Evaluation (APACHE) consists of a groups of equations used for predicting outcomes in critically ill patients. See: http://eicu-crd.mit.edu/eicutables/apachePredVar/

In [0]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apachepredvar`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apachepredvar = run_query(query)

In [0]:
apachepredvar.columns

## Questions

- Was the patient ventilated during (APACHE) day 1 of their stay?
- Is the patient recorded as having diabetes?

# `apachepatientresult` table

The `apachepatientresult` table provides predictions made by the APACHE score (versions IV and IVa), including probability of mortality, length of stay, and ventilation days. See: http://eicu-crd.mit.edu/eicutables/apachePatientResult/

In [0]:
# set the where clause to select the stay of interest
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apachepatientresult`
WHERE patientunitstayid = {}
""".format(patientunitstayid)

apachepatientresult = run_query(query)

In [0]:
apachepatientresult

## Questions

- What versions of the APACHE score are computed?
- How many days during the stay was the patient ventilated?
- How long was the patient predicted to stay in hospital?
- Was this prediction close to the truth?